In [23]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix, eye
from scipy.sparse.linalg import spsolve, inv
from scipy.stats import truncnorm
from tqdm.notebook import tqdm
from scipy.integrate import solve_ivp
import numba as nb
from matplotlib.gridspec import GridSpec
from ast import literal_eval
import pandas as pd
from plotstyle import defaultStyle, save_fig


def elliptic_matrix(n, bc="zero_flux"): 
    row_ind = [i for i in range(n**2-1) if not i % n == n-1] + [i for i in range(1, n**2) if not i % n == 0] + list(range(n**2-n)) + list(range(n, n**2))
    col_ind = [i for i in range(1, n**2) if not i % n == 0] + [i for i in range(n**2-1) if not i % n == n-1] + list(range(n, n**2)) + list(range(n**2-n))
    data = [-1] * (n-1) * n * 2 + [-1] * (n**2*2 - 2*n)
    A_arr = csc_matrix((data, (row_ind, col_ind)), shape=(n**2, n**2))
    if bc == "zero":
        A_arr_diag_compensation == 0
    elif bc == "zero_flux":
        A_arr_diag_compensation = np.array(np.sum(A_arr, axis=1)).flatten()
    row_ind += list(range(n**2))
    col_ind += list(range(n**2))
    data += list(np.array([4] * n**2) - A_arr_diag_compensation - 4)
    A_arr = csc_matrix((data, (row_ind, col_ind)), shape=(n**2, n**2))
    return A_arr

In [66]:
nGrid = 100
gamma = 0.1
D = 0.1
a = 1 # gamma
a_std = a * 1
thresh = 5
A_arr = elliptic_matrix(nGrid)
A_arr += gamma / D * eye(nGrid**2)

css = []
crs = []
for sr in np.arange(0.1, 1, 0.1):
    s = np.zeros(nGrid ** 2)
    m = int(sr * nGrid ** 2)
    s[np.random.choice(np.arange(nGrid ** 2), m, replace=False)] = 1
    sol = spsolve(A_arr, s / D)
    css.append(np.mean(sol[s==1]))
    crs.append(np.mean(sol[s==0]))
p = np.polyfit(css, crs, deg=1)
p

array([ 0.99825085, -2.56005051])

In [56]:
nGrid = 3
gamma = 1
D = 10
a = 1 # gamma
a_std = a * 1
thresh = 5
A_arr = elliptic_matrix(nGrid)
A_arr += gamma / D * eye(nGrid**2)
A_arr = A_arr.toarray()
A_inv = np.linalg.inv(A_arr)
print(np.linalg.eigvals(A_arr))
print(np.trace(A_inv))

[6.1 0.1 2.1 1.1 1.1 4.1 4.1 3.1 3.1]
13.59127288897316


In [51]:
np.linalg.det(A_arr)

8.881784197001244e-16

In [52]:
A_arr

array([[ 2., -1., -1.,  0.],
       [-1.,  2.,  0., -1.],
       [-1.,  0.,  2., -1.],
       [ 0., -1., -1.,  2.]])